In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
from ase import Atom, Atoms
from ase.data import chemical_symbols, covalent_radii, vdw_alvarez
from ase.io import read, write
from pymatgen.core import Element
from scipy import stats
from tqdm.auto import tqdm

from mlip_arena.models.utils import REGISTRY, MLIPEnum

# model_name = "MACE-MP(M)"

# calc = MLIPEnum[model_name].value()

In [ ]:
for model in MLIPEnum:
    
    model_name = model.name
    
    if model_name == 'EquiformerV2(OC22)':
        continue
    
    json_fpath = Path(REGISTRY[model_name]["family"]) / "homonuclear-diatomics.json"
    
    if json_fpath.exists():
        continue
    
    print(f"========== {model_name} ==========")

    calc = MLIPEnum[model_name].value()

    for symbol in tqdm(chemical_symbols[1:]):

        s = set([symbol])

        if "X" in s:
            continue

        try:
            atom = Atom(symbol)
            rmin = 0.9 * covalent_radii[atom.number]
            rvdw = vdw_alvarez.vdw_radii[atom.number] if atom.number < len(vdw_alvarez.vdw_radii) else np.nan
            rmax = 3.1 * rvdw if not np.isnan(rvdw) else 6
            rstep = 0.01

            a = 2 * rmax

            npts = int((rmax - rmin)/rstep)

            rs = np.linspace(rmin, rmax, npts)
            es = np.zeros_like(rs)

            da = symbol + symbol

            out_dir = Path(REGISTRY[model_name]["family"]) / str(da)
            os.makedirs(out_dir, exist_ok=True)

            skip = 0

            element = Element(symbol)

            try:
                m = element.valence[1]
                if element.valence == (0, 2):
                    m = 0
            except:
                m = 0


            r = rs[0]

            positions = [
                [a/2-r/2, a/2, a/2],
                [a/2+r/2, a/2, a/2],
            ]

            traj_fpath = out_dir / f"{model_name}.extxyz"

            if traj_fpath.exists():
                traj = read(traj_fpath, index=":")
                skip = len(traj)
                atoms = traj[-1]
            else:
                # Create the unit cell with two atoms
                atoms = Atoms(
                    da,
                    positions=positions,
                    # magmoms=magmoms,
                    cell=[a, a+0.001, a+0.002],
                    pbc=True
                )

            print(atoms)

            atoms.calc = calc

            for i, r in enumerate(tqdm(rs)):

                if i < skip:
                    continue

                positions = [
                    [a/2-r/2, a/2, a/2],
                    [a/2+r/2, a/2, a/2],
                ]

                # atoms.set_initial_magnetic_moments(magmoms)

                atoms.set_positions(positions)

                es[i] = atoms.get_potential_energy()

                write(traj_fpath, atoms, append="a")
        except Exception as e:
            print(e)


========== EquiformerV2(OC20) ==========


/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/fairchem/core/modules/normalization/normalizer.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "mean": torch.tensor(state_dict["mean"]),


  0%|          | 0/118 [00:00<?, ?it/s]

Atoms(symbols='H2', pbc=True, cell=[7.4399999999999995, 7.441, 7.441999999999999])


  0%|          | 0/344 [00:00<?, ?it/s]

...

  0%|          | 0/623 [00:00<?, ?it/s]

Atoms(symbols='Nb2', pbc=True, cell=[15.872000000000002, 15.873000000000001, 15.874000000000002])


  0%|          | 0/646 [00:00<?, ?it/s]

Atoms(symbols='Mo2', pbc=True, cell=[15.190000000000001, 15.191, 15.192000000000002])


  0%|          | 0/620 [00:00<?, ?it/s]

In [3]:


for model in MLIPEnum:
    
    model_name = model.name
    
    print(f"========== {model_name} ==========")
    
    df = pd.DataFrame(columns=[
        "name", 
        "method", 
        "R", "E", "F", "S^2", 
        "force-flip-times",
        "force-total-variation",
        "energy-diff-flip-times",
        "energy-grad-norm-max",
        "energy-jump",
        "energy-total-variation",
        "conservation-deviation",
        "spearman-descending-force",
        "spearman-ascending-force",
        "spearman-repulsion-energy",
        "spearman-attraction-energy"
    ])
    

    for symbol in tqdm(chemical_symbols[1:]):

        da = symbol + symbol

        out_dir = Path(REGISTRY[model_name]["family"]) / da

        traj_fpath = out_dir / f"{model_name}.extxyz"


        if traj_fpath.exists():
            traj = read(traj_fpath, index=":")
        else:
            continue

        Rs, Es, Fs, S2s = [], [], [], []
        for atoms in traj:

            vec = atoms.positions[1] - atoms.positions[0]
            r = np.linalg.norm(vec)
            e = atoms.get_potential_energy()
            f = np.inner(vec/r, atoms.get_forces()[1])
            # s2 = np.mean(np.power(atoms.get_magnetic_moments(), 2))

            Rs.append(r)
            Es.append(e)
            Fs.append(f)
            # S2s.append(s2)

        rs = np.array(Rs)
        es = np.array(Es)
        fs = np.array(Fs)

        indices = np.argsort(rs)[::-1]
        rs = rs[indices]
        es = es[indices]
        fs = fs[indices]

        iminf = np.argmin(fs)
        imine = np.argmin(es)

        de_dr = np.gradient(es, rs)
        d2e_dr2 = np.gradient(de_dr, rs)

        rounded_fs = np.copy(fs)
        rounded_fs[np.abs(rounded_fs) < 1e-2] = 0
        fs_sign = np.sign(rounded_fs)
        fs_sign = fs_sign[fs_sign != 0]

        # rounded_ediff = np.diff(es)
        # rounded_ediff[np.abs(rounded_ediff) < zero_threshold] = 0
        ediff = np.diff(es)
        ediff[np.abs(ediff) < 1e-3] = 0
        ediff_sign = np.sign(ediff)
        mask = ediff_sign != 0
        ediff = ediff[mask]
        ediff_sign = ediff_sign[mask]
        ediff_flip = np.diff(ediff_sign) != 0
        ejump = np.abs(ediff[:-1][ediff_flip]).sum() + np.abs(ediff[1:][ediff_flip]).sum()

        conservation_deviation = np.mean(np.abs(fs + de_dr))

        data = {
            "name": da,
            "method": model_name,
            "R": rs,
            "E": es,
            "F": fs,
            "S^2": S2s,
            "force-flip-times": np.sum(np.diff(fs_sign)!=0),
            "force-total-variation": np.sum(np.abs(np.diff(fs))),
            "energy-diff-flip-times": np.sum(ediff_flip),
            "energy-grad-norm-max": np.max(np.abs(de_dr)),
            "energy-jump": ejump,
            # "energy-grad-norm-mean": np.mean(de_dr_abs),
            "energy-total-variation": np.sum(np.abs(np.diff(es))),
            "conservation-deviation": conservation_deviation,
            "spearman-descending-force": stats.spearmanr(rs[iminf:], fs[iminf:]).statistic,
            "spearman-ascending-force": stats.spearmanr(rs[:iminf], fs[:iminf]).statistic,
            "spearman-repulsion-energy": stats.spearmanr(rs[imine:], es[imine:]).statistic,
            "spearman-attraction-energy": stats.spearmanr(rs[:imine], es[:imine]).statistic,
        }

        df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)


    json_fpath = Path(REGISTRY[model_name]["family"]) / "homonuclear-diatomics.json"

    if json_fpath.exists():
        df0 = pd.read_json(json_fpath)
        df = pd.concat([df0, df], ignore_index=True)
        df.drop_duplicates(inplace=True, subset=["name", "method"], keep='last')

    df.to_json(json_fpath, orient="records")

========== MACE-MP(M) ==========


  0%|          | 0/118 [00:00<?, ?it/s]

/tmp/ipykernel_768993/102531167.py:107: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
/tmp/ipykernel_768993/102531167.py:101: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  "spearman-descending-force": stats.spearmanr(rs[iminf:], fs[iminf:]).statistic,
/tmp/ipykernel_768993/102531167.py:103: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  "spearman-repulsion-energy": stats.spearmanr(rs[imine:], es[imine:]).statistic,


========== CHGNet ==========


  0%|          | 0/118 [00:00<?, ?it/s]

...

========== MACE-OFF(M) ==========


  0%|          | 0/118 [00:00<?, ?it/s]

========== ALIGNN ==========


  0%|          | 0/118 [00:00<?, ?it/s]

OSError: Cannot save file into a non-existent directory: 'alignn'

In [12]:
df

,name,method,R,E,F,S^2,force-flip-times,force-total-variation,energy-diff-flip-times,energy-grad-norm-max,energy-jump,energy-total-variation,conservation-deviation,spearman-descending-force,spearman-ascending-force,spearman-repulsion-energy,spearman-attraction-energy
89,HH,MACE-MP(M),"[3.7199999999999998, 3.70996794, 3.69993586, 3...","[-2.348365306854248, -2.3483591079711914, -2.3...","[1.7e-07, 0.00035892, 0.00068535, 0.00112034, ...",[],3,80.309463,2,65.875268,0.009835,15.816339,0.043480,-0.999421,0.840215,-1.000000,0.794565
90,HeHe,MACE-MP(M),"[4.433, 4.4229736200000005, 4.41294724, 4.4029...","[0.022882699966430664, 0.022882461547851562, 0...","[0.0, -0.00010788, -0.00021458, -0.00032269, -...",[],2,1.485210,0,0.821236,0.000000,0.344755,0.001872,-0.904433,0.077295,-1.000000,0.093575
91,LiLi,MACE-MP(M),"[6.572000000000001, 6.561981520000001, 6.55196...","[-1.8418288230895996, -1.8418288230895996, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],1,8.583572,1,8.122311,0.002224,5.332280,0.013703,-0.999858,0.993021,-0.999996,0.995153
92,BeBe,MACE-MP(M),"[6.138000000000001, 6.12797338, 6.117946759999...","[-0.6769685745239258, -0.6769685745239258, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],2,29.133944,2,10.346194,0.016283,6.239225,0.022079,NaN,-0.044580,-0.998691,0.999892
93,BB,MACE-MP(M),"[5.921000000000001, 5.91097088, 5.900941739999...","[-1.404820442199707, -1.404820442199707, -1.40...","[-6e-08, -0.00011319, -0.00023184, -0.0003534,...",[],2,330.078666,2,123.581618,0.543755,40.039458,0.138081,NaN,0.201590,-0.999921,0.999947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,ThTh,MACE-MP(M),"[9.082999999999998, 9.07297364, 9.062947300000...","[-4.095590591430664, -4.095590591430664, -4.09...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],2,16.226657,2,6.413536,0.006823,3.392294,0.024297,NaN,0.026348,-0.997169,0.918191
174,PaPa,MACE-MP(M),"[8.927999999999999, 8.91797468, 8.90794936, 8....","[-8.707170486450195, -8.707170486450195, -8.70...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],2,6.407403,2,2.536645,0.006878,2.349456,0.018977,-0.627256,0.460505,NaN,0.950999
175,UU,MACE-MP(M),"[8.401, 8.390974320000002, 8.38094864, 8.37092...","[-11.437246322631836, -11.437246322631836, -11...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],1,34.878494,1,25.824510,0.004967,11.734995,0.025810,-0.974165,0.939272,-1.000000,0.966465
176,NpNp,MACE-MP(M),"[8.741999999999999, 8.7319829, 8.72196582, 8.7...","[-18.34619903564453, -18.34619903564453, -18.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],3,14.386345,3,8.632403,0.010139,2.824085,0.015532,-1.000000,0.189479,-1.000000,0.439673
